<a href="https://colab.research.google.com/github/Raj-Harshit23/CSV-Grade-File-Manager/blob/main/BM-BSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git

  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-46om5q92
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-46om5q92
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


In [ ]:
import math
def CRRTree(K,T,S,sig,r,N,PorC):

    dt=T/N;
    dxu=math.exp(sig*math.sqrt(dt));
    dxd=math.exp(-sig*math.sqrt(dt));
    pu=((math.exp(r*dt))-dxd)/(dxu-dxd);
    pd=1-pu;
    disc=math.exp(-r*dt);

    St = [0] * (N+1)
    C = [0] * (N+1)

    St[0]=S*dxd**N;

    for j in xrange(1, N+1):
        St[j] = St[j-1] * dxu/dxd;

    for j in xrange(1, N+1):
        if PorC == 'p':
            C[j] = max(K-St[j],0);
        elif PorC == 'c':
            C[j] = max(St[j]-K,0);

    for i in xrange(N, 0, -1):
        for j in xrange(0, i):
            C[j] = disc*(pu*C[j+1]+pd*C[j]);

    return C[0]

In [ ]:
import math

def fnor2(x):

    y=0.5*(1+math.erf(x/math.sqrt(2)));

    return y

def BS(t,St,K,T,r,sig,PorC):

    Tmt=T-t;
    ATmt=sig*math.sqrt(Tmt);
    logo=math.log(St/K);
    Ap=(logo+(r+0.5*sig**2)*Tmt)/ATmt;
    An=Ap-ATmt;

    if PorC == 'c':
        p=St*fnor2(Ap)-K*math.exp(-r*Tmt)*fnor2(An);
    elif PorC == 'p':
        p=K*math.exp(-r*Tmt)*fnor2(-An)-St*fnor2(-Ap);

    return p

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval ##used to get the historical data
import random
## top 10 stocks
n50=np.array(['NIFTY','HINDALCO','MARUTI','NESTLEIND', 'ONGC', 'TATAMOTORS','ITC','SUNPHARMA','BHARATIARTL','CIPLA'])
result=pd.DataFrame()
for b in range(0,3):
  tv=TvDatafeed()
  data=tv.get_hist(symbol=n50[b],exchange='NSE',interval=Interval.in_daily,n_bars=1100)
  df=pd.DataFrame(data)
  vd=252 ##offset for volatility
  rf=0.0695  ## risk free rate
  df=df.drop(['symbol','high','low','volume'],axis=1)
  df.reset_index(inplace=True)
  df['datetime']=pd.to_datetime(df['datetime'])
  df['day_of_week']=df['datetime'].dt.day_of_week ## if day of week is 3, it is thursday
  df['date']=df['datetime'].dt.day
  df['daily_return']=np.log(df['close']/df['open'])
  #print(df.head())
  act_index=[]
  for i in range(1080-vd):
    if df.loc[i+vd,'date'] > df.loc[i+vd+1,'date']:
      flag=False
      for j in range(5):
        if df.loc[i+vd-j,'day_of_week'] == 3:
          act_index.append(i+vd-j)
          flag=True
          break
      ## incase the given thursadayis a holiday, we would check on the friday. this ensures that we dont skip any month
      ## due to lack of thursday data in the dataframe
      if flag==False:
        for j in range(5):
          if df.loc[i+vd-j,'day_of_week'] == 4:
            act_index.append(i+vd-j)
            flag=True
            break
  #print(len(act_index))
  act_index=act_index[:35]
  #print(df.loc[act_index])
  #print(df[410:420])
  ## now calculating for each index/month
  for e in range(39):
    start_index=act_index[e]
    start_price=df.loc[start_index,'open']
    vol=df.loc[start_index-252:start_index,'daily_return']
    vol=vol.to_numpy()
    sigma=vol.std()*np.sqrt(252)
    print(sigma)

    for num_of_sps in range(-3,4):
      strike_price=start_price+num_of_sps*50
      bs_premium=BS(0,start_price,strike_price,1.0/12,rf,sigma,'c')
      bt_premium=CRRTree(strike_price,1.0/12,start_price,sigma,rf,1000,'c')
      market_

40
               datetime       open      close  day_of_week  date  daily_return
263 2021-02-25 03:45:00  15079.850  15097.350            3    25      0.001160
282 2021-03-25 03:45:00  14570.900  14324.900            3    25     -0.017027
303 2021-04-29 03:45:00  14979.000  14894.900            3    29     -0.005630
322 2021-05-27 03:45:00  15323.950  15337.850            3    27      0.000907
342 2021-06-24 03:45:00  15737.300  15790.450            3    24      0.003372
366 2021-07-29 03:45:00  15762.700  15778.450            3    29      0.000999
385 2021-08-26 03:45:00  16627.949  16636.900            3    26      0.000538
409 2021-09-30 03:45:00  17718.900  17618.150            3    30     -0.005702
428 2021-10-28 03:45:00  18187.650  17857.250            3    28     -0.018333
446 2021-11-25 03:45:00  17417.301  17536.250            3    25      0.006806
471 2021-12-30 03:45:00  17201.449  17203.949            3    30      0.000145
490 2022-01-27 03:45:00  17062.000  17110.150    

40
               datetime       open      close  day_of_week  date  daily_return
263 2021-02-25 03:45:00  337.60001  350.70001            3    25      0.038069
282 2021-03-25 03:45:00  319.04999  314.00000            3    25     -0.015955
303 2021-04-29 03:45:00  367.39999  372.14999            3    29      0.012846
322 2021-05-27 03:45:00  382.04999  385.85001            3    27      0.009897
342 2021-06-24 03:45:00  369.00000  369.04999            3    24      0.000135
366 2021-07-29 03:45:00  420.50000  458.10001            3    29      0.085643
385 2021-08-26 03:45:00  431.50000  424.10001            3    26     -0.017298
409 2021-09-30 03:45:00  495.25000  487.95001            3    30     -0.014850
428 2021-10-28 03:45:00  455.85001  468.79999            3    28      0.028012
446 2021-11-25 03:45:00  451.50000  447.04999            3    25     -0.009905
471 2021-12-30 03:45:00  454.00000  449.64999            3    30     -0.009628
490 2022-01-27 03:45:00  482.60001  489.14999    